In [1]:
import os

# Hàm xác định loại file theo phần mở rộng
def get_file_type(ext):
    ext = ext.lower()
    if ext == ".html":
        return "HTML"
    elif ext == ".css":
        return "CSS"
    elif ext == ".js":
        return "JavaScript"
    elif ext == ".php":
        return "PHP"
    elif ext in [".jpg", ".jpeg", ".png", ".gif", ".svg", ".webp", ".bmp", ".ico", ".tiff"]:
        return "Image"
    elif ext in [".woff", ".woff2", ".ttf", ".eot", ".otf"]:
        return "Font"
    elif ext in [".mp4", ".webm", ".ogg", ".avi", ".mov"]:
        return "Video"
    elif ext in [".mp3", ".wav", ".aac", ".flac"]:
        return "Audio"
    elif ext in [".json", ".xml", ".csv", ".yml", ".map", ".txt", ".md"]:
        return "Text/Other"
    elif ext in [".zip", ".rar", ".7z", ".tar", ".gz"]:
        return "Archive"
    else:
        return "Other"

# Hàm duyệt toàn bộ thư mục
def list_all_files(folder_path):
    print(f"\n📂 Đang quét thư mục: {folder_path}\n")
    for root, dirs, files in os.walk(folder_path):
        for file in files:
            full_path = os.path.join(root, file)
            rel_path = os.path.relpath(full_path, folder_path)
            ext = os.path.splitext(file)[1]
            file_type = get_file_type(ext)
            size_kb = round(os.path.getsize(full_path) / 1024, 2)
            print(f"[{file_type:<12}] {rel_path:<70} | {size_kb:>8} KB")

# 👉 Đổi đường dẫn này thành thư mục UI thực tế của bạn
list_all_files(r"C:\Users\ADMIN\OneDrive\Máy tính\MyResume")



📂 Đang quét thư mục: C:\Users\ADMIN\OneDrive\Máy tính\MyResume

[Other       ] .env                                                                   |     0.16 KB
[Other       ] app.py                                                                 |     2.64 KB
[Other       ] check_bug.ipynb                                                        |    49.52 KB
[Other       ] google_search.py                                                       |     1.14 KB
[Text/Other  ] langchain-2025-a4bdd-462307-649b4a58d244.json                          |     2.34 KB
[Other       ] llama-3.2-3b-it-ecommerce-chatbot-q4_k_m.gguf                          | 1972049.25 KB
[Text/Other  ] myresume\Readme.txt                                                    |     0.21 KB
[PHP         ] myresume\forms\contact.php                                             |     1.32 KB
[Text/Other  ] myresume\forms\Readme.txt                                              |     0.17 KB
[CSS         ] static\assets\css\

In [1]:
import os
print(os.getcwd())


c:\Users\ADMIN\OneDrive\Máy tính\MyResume


In [2]:
import os

def print_structure(base_dir):
    for root, dirs, files in os.walk(base_dir):
        level = root.replace(base_dir, "").count(os.sep)
        indent = "    " * level
        print(f"{indent}📁 {os.path.basename(root)}/")
        for file in files:
            print(f"{indent}    📄 {file}")

# Đặt đường dẫn thư mục gốc của bạn tại đây:
base_path = r"C:\Users\ADMIN\OneDrive\Máy tính\MyResume"
print_structure(base_path)


📁 MyResume/
    📄 app.py
    📄 check_bug.ipynb
    📄 llama-3.2-3b-it-ecommerce-chatbot-q4_k_m.gguf
    📁 MyResume/
        📄 Readme.txt
        📁 forms/
            📄 contact.php
            📄 Readme.txt
        📁 static/
            📁 assets/
                📁 css/
                    📄 main.css
                📁 img/
                    📄 1721737589608.gif
                    📄 apple-touch-icon.png
                    📄 favicon.png
                    📄 hero-bg.jpg
                    📄 llm-background.jpg
                    📄 profile-img.jpg
                    📄 services.jpg
                    📁 masonry-portfolio/
                        📄 masonry-portfolio-1.jpg
                        📄 masonry-portfolio-2.jpg
                        📄 masonry-portfolio-3.jpg
                        📄 masonry-portfolio-4.jpg
                        📄 masonry-portfolio-5.jpg
                        📄 masonry-portfolio-6.jpg
                        📄 masonry-portfolio-7.jpg
                       

In [ ]:
from flask import Flask, request, jsonify, render_template
from llama_cpp import Llama
from datetime import datetime
import gspread

app = Flask(__name__)

# 1️⃣ Kết nối Google Sheets (nếu muốn lưu lịch sử)
gc = gspread.service_account(filename="langchain-2025-a4bdd-462307-649b4a58d244.json")
sheet_url = "https://docs.google.com/spreadsheets/d/1b_Tt-iUaR_xjmmq0dJnx7xZ2GYVzZiGBi1mkshCCWZc"
sheet = gc.open_by_url(sheet_url).sheet1

# 2️⃣ Load mô hình LLaMA
llm = Llama(
    model_path="llama-3.2-3b-it-ecommerce-chatbot-q4_k_m.gguf",
    n_ctx=2048,
    n_threads=6,
    temperature=0.7,
    stop=["User:", "Lytch:"]
)

# 3️⃣ System prompt (vai trò)
SYSTEM_PROMPT = """
You are a top-rated customer service agent named Lytch.
Be polite, clear, and concise.
When listing steps or multiple items, please format them using:
- Bullet points, or
- Numbered lists (1., 2., 3.)
Add line breaks (\\n) between each step for readability.
Avoid writing long, dense paragraphs.
"""

# 4️⃣ Route trang chính
@app.route("/")
def index():
    return render_template("index.html")

# 5️⃣ Route API Chatbot
@app.route("/chat", methods=["POST"])
def chat():
    user_input = request.json.get("message", "").strip()
    if not user_input:
        return jsonify({"reply": "❗ Bạn chưa nhập gì cả."})

    # Tạo prompt
    prompt = SYSTEM_PROMPT + f"\nUser: {user_input}\nLytch:"

    # Gọi mô hình sinh câu trả lời
    response = llm.create_completion(
        prompt=prompt,
        max_tokens=512,
        echo=False
    )

    raw_output = response["choices"][0]["text"].strip()
    if "User:" in raw_output:
        bot_reply = raw_output.split("User:")[0].strip()
    else:
        bot_reply = raw_output

    # Ghi vào Google Sheet (nếu cần)
    timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    sheet.append_row([timestamp, user_input, bot_reply])

    return jsonify({"reply": bot_reply})


if __name__ == "__main__":
    app.run(debug=True)


In [3]:
from sentence_transformers import SentenceTransformer

# Load the model from a local folder (replace path if needed)
model = SentenceTransformer("all-MiniLM-L6-v2")

# Example sentence
sentence = "I want to know the return policy."

# Encode the sentence into a dense vector
embedding = model.encode(sentence)

# Check the shape of the resulting embedding
print("Embedding shape:", embedding.shape)


Embedding shape: (384,)


In [2]:
import importlib

# Danh sách các thư viện cần thiết
required_packages = {
    "flask": "Flask==2.3.3",
    "llama_cpp": "llama-cpp-python==0.2.11",
    "gspread": "gspread==5.12.0",
    "dotenv": "python-dotenv==1.0.1",
    "requests": "requests==2.31.0"
}

missing = []

print("🔍 Đang kiểm tra các thư viện cần thiết...\n")

for module, pip_name in required_packages.items():
    try:
        importlib.import_module(module)
        print(f"✅ Đã cài: {pip_name}")
    except ImportError:
        print(f"❌ Thiếu: {pip_name}")
        missing.append(pip_name)

if missing:
    print("\n📦 Bạn có thể cài bằng lệnh:")
    print("pip install " + " ".join(missing))
else:
    print("\n🎉 Tất cả thư viện đã sẵn sàng!")


🔍 Đang kiểm tra các thư viện cần thiết...

✅ Đã cài: Flask==2.3.3
✅ Đã cài: llama-cpp-python==0.2.11
✅ Đã cài: gspread==5.12.0
✅ Đã cài: python-dotenv==1.0.1
✅ Đã cài: requests==2.31.0

🎉 Tất cả thư viện đã sẵn sàng!
